# 2D Poisson Equation Solver

We use the recursion based exploration policy to find the closed-form solution of 2d Poisson equation.

In [1]:
import numpy as np
import pickle
from ssde import PDERecursionSolver
import sympy as sp
from ssde.utils import rect

# Generate some data on the bcs
np.random.seed(10)

real_expression_dict = {
    'nguyen-9': "sin(x1)+sin(x2**2)",
    'nguyen-10': "2*sin(x1)*cos(x2)",
    'nguyen-11': "x1**x2",
    'nguyen-12': "x1**4 - x1**3 + 0.5 * x2**2 -x2",
    'jin-1': "2.5*x1**4 - 1.3*x1**3 + 0.5*x2**2 - 1.7*x2"
}

domain_dict = {
    'nguyen-9': [0.5, 1.5],
    'nguyen-10': [0.5, 1.5],
    'nguyen-11': [0.5, 1.5],
    'nguyen-12': [-1, 1],
    'jin-1': [-1,1]
}


test_case = 'nguyen-9'
solution = sp.sympify(real_expression_dict[test_case])
left_bc, right_bc = domain_dict[test_case]

def calculate_soure(solution):
    ''' Calculate the source term of the PDE '''
    # solution: sympy expr of the solution
    real_params = dict()
    for symbol in sp.preorder_traversal(solution):
        if isinstance(symbol, sp.Symbol):
            exec('%s = sp.Symbol("%s")' % (symbol.name, symbol.name))
            if symbol.name not in real_params:
                real_params[symbol.name] = None
    real_params = sorted(list(real_params.keys()))
    print(f'real_params:{real_params}')
    source = 0
    for i in real_params:
        source += sp.diff(solution, i, 2)
    print(f'source:{source}')
    solution_func = sp.lambdify(real_params, solution, modules='numpy')
    source_func = sp.lambdify(real_params, source, modules='numpy')
    return solution_func, source_func, real_params

solution_func, source_func, real_params = calculate_soure(solution)

real_params:['x1', 'x2']
source:2*(-2*x2**2*sin(x2**2) + cos(x2**2)) - sin(x1)


## Expression of x1

We first generate the 1D data for the 2D Poisson equation. It is only used to find the expression vs. x1 while x2 is viewed as parameters.

In [2]:
# The second-order derivative of x2 is calculated by finite difference with respect to the boundary conditions in the x2 direction
# Then the first-order derivative of x1 in the boundary direction is derived.
n_x1bc, n_x2_bc = 2, 51
d_x2 = (right_bc - left_bc)/(n_x2_bc-1)
X1_bc = np.array([left_bc,right_bc]).reshape(2,1)
X2_bc_all = np.linspace(left_bc, right_bc, n_x2_bc).reshape(n_x2_bc,1)

y_bc_all = []
source_bc_all = []
for i in X2_bc_all:
    temp = np.tile(i, (n_x1bc, 1))
    X_bc_temp = np.concatenate([X1_bc, temp], axis=1).transpose(1, 0)
    y_bc_all.append(solution_func(*X_bc_temp).reshape(-1, 1))
    source_bc_all.append(source_func(*X_bc_temp).reshape(-1, 1))
X_bc1 = X1_bc
y_bc_all = np.concatenate(y_bc_all, axis=1)
source_bc_all = np.concatenate(source_bc_all, axis=1)
# Convolution along the column direction calculates the second-order derivative of x2
y_dx2_all = []
for i in range(y_bc_all.shape[0]):
    y_dx2_all.append(np.convolve(y_bc_all[0,:], [1, -2, 1], 'valid').reshape(1,-1)/d_x2**2)
y_dx2_all = np.concatenate(y_dx2_all, axis=0)
# Calculate the first derivative of x1
y_dx1_all = source_bc_all[:,1:-1] - y_dx2_all
# Randomly select 10 points from y_bc_all, with an index range of (1,49) for a total of 49 points
n_samples = 10
index = np.random.choice(np.arange(1,n_x2_bc-1), n_samples, replace=False)
y_bc1 = np.concatenate([y_dx1_all[:,index-1],y_bc_all[:,index]], axis=0)

In [3]:
n_x1bc, n_x2_bc = 10, 2
X1_bc = np.random.uniform(left_bc, right_bc, (n_x1bc,1))
X2_bc = np.array([left_bc,right_bc]).reshape(2,1)

y_bc2 = []
for i in X2_bc:
    temp = np.tile(i, (n_x1bc, 1))
    X_bc_temp = np.concatenate([X1_bc, temp], axis=1).transpose(1, 0)
    y_bc2.append(solution_func(*X_bc_temp).reshape(-1, 1))
X_bc2 = X1_bc
y_bc2 = np.concatenate(y_bc2, axis=1)

# Boundary conditions in the x2 direction, used for refine
# the first half of the rows are  the obtained second-order differences of x1(on the border)
# and the second half are directly values
print(f'X_bc1: {X_bc1.shape}')
print(f'y_bc1: {y_bc1.shape}')
# Boundary conditions in the x1 direction, used for initial selection
print(f'X_bc2: {X_bc2.shape}')
print(f'y_bc2:{y_bc2.shape}')


X_input = [X_bc1, X_bc2]
y_input = [y_bc1, y_bc2]

X_bc1: (2, 1)
y_bc1: (4, 10)
X_bc2: (10, 1)
y_bc2:(10, 2)


In [4]:
import time
from ssde.utils import jupyter_logging
import os
if not os.path.exists(f'./logs/{test_case}'):
    os.makedirs(f'./logs/{test_case}')
if os.path.exists(f"./logs/{test_case}/poisson2d_01.log"):
    os.remove(f"./logs/{test_case}/poisson2d_01.log")
@jupyter_logging(f"./logs/{test_case}/poisson2d_01.log")
def solve_pde(X_input, y_input):
    x1_model = PDERecursionSolver(f"./configs/{test_case}/config_poisson_gp.json")
    start_time = time.time()
    config = x1_model.fit(X_input, y_input) # Should solve in ~10 hours
    print(f'Time used(s): {time.time()-start_time}')
    x1_traversal = config.program_.traversal
    x1_exp = config.program_.sympy_expr
    print('Identified var x1\'s parametirc expression:')
    print(x1_exp)
    print('Identified var x1\'s traversal:')
    print(x1_traversal)
    return x1_traversal, x1_exp, x1_model

x1_traversal, x1_exp, x1_model = solve_pde(X_input, y_input)
x1_model.save(f'./models/{test_case}/poisson2d_x1_model')
print('x1 model successfully saved!')
with open(f'./models/{test_case}/x1_traversal_poisson2d.pkl', 'wb') as f:
    pickle.dump(x1_traversal, f)
    print('Successfully saved x1_traversal!')


Saved Trainer state to ./models/nguyen-9/poisson2d_x1_model/trainer.json.
x1 model successfully saved!
Successfully saved x1_traversal!


## Expression of x2

From the 1D expression, we can get the value of parameters expressed by `Nxexpr` using constants optimization with least squares method.
And then we can use it to explore the expression of x2.

In [5]:
# optimize the constants in  x1_traversal to get the next parametric expression of x2
# TBD: Using a network to substitute the optimization
from scipy.optimize import least_squares
from ssde.execute import cython_recursion_execute as ce
from ssde.program import Program
# Compute the value of const (actually the label of x2)
n_boundary = 20
X_bc = rect(xmin=[left_bc, left_bc],
                    xmax=[right_bc, right_bc],
                    n=n_boundary,
                    method='uniform')
y_bc = solution_func(*X_bc.T).reshape(-1, 1)
print(f'shape of y_boundary:{y_bc.shape}')

with open(f'./models/{test_case}/x1_traversal_poisson2d.pkl', 'rb') as f:
    x1_traversal = pickle.load(f)
test_p = Program()
test_p.traversal = x1_traversal
print("Identified var x1's parametirc expression:")
print(test_p.sympy_expr)

def opti_consts(nxexpr):
    for token in x1_traversal:
        if token.name == 'Nxexpr':
            token.value = nxexpr.reshape(-1,1)
    y_bc_hat = ce(x1_traversal, X_bc)
    return (y_bc_hat-y_bc).ravel()

consts = np.ones(n_boundary)
res = least_squares(opti_consts, consts, method='lm')
# You can also test the optimization is correct or not
# consts_real = (0.5 * X_bc[:,1]**2 - 1.7 * X_bc[:,1])
# print('abs error:', np.abs(consts_real-res.x).mean())
X_bc = X_bc
y_bc = np.array(res.x, dtype=np.float64).reshape(-1,1)
print("Shape of label vs. x2:", y_bc.shape)

shape of y_boundary:(20, 1)
Identified var x1's parametirc expression:
Nxexpr + sin(x1)
Shape of label vs. x2: (20, 1)


In [6]:
import time
from ssde.utils import jupyter_logging

# Fit the model
if os.path.exists(f"./logs/{test_case}/poisson2d_02.log"):
    os.remove(f"./logs/{test_case}/poisson2d_02.log")
@jupyter_logging(f"./logs/{test_case}/poisson2d_02.log")
def solve_pde_intervar(X_input, y_input, model=None):
    start_time = time.time()
    if model is not None:
        x2_model = model
    else:
        x2_model = PDERecursionSolver(f"./configs/{test_case}/config_poisson_gp.json")
    x2_model = x2_model.fit(X_input, y_input, start_n_var=2) # Should solve in ~10 hours
    print(f'Time used(s): {time.time()-start_time}')
    x2_traversal = x2_model.program_.traversal
    x2_exp = x2_model.program_.sympy_expr
    print('Identified var x2\'s parametirc expression:')
    print(x2_exp)
    print('Identified var x2\'s traversal:')
    print(x2_traversal)
    return x2_traversal, x2_exp, x2_model


X_input = [None, X_bc]
y_input = [None, y_bc]
x2_traversal, x2_exp, x2_model = solve_pde_intervar(X_input, y_input)
with open(f'./models/{test_case}/x2_traversal_poisson2d.pkl', 'wb') as f:
    pickle.dump(x2_traversal, f)
    print('Successfully saved x2_traversal!')
x2_model.save(f'./models/{test_case}/poisson2d_x2_model')

Successfully saved x2_traversal!
Saved Trainer state to ./models/nguyen-9/poisson2d_x2_model/trainer.json.


## Expression of x1 and x2

Finally, we can get the closed-form solution of 2D Poisson equation with combined expression of x1 and x2. And we can refine the constants in the expression using the pde loss as the target.

In [7]:
import pickle
import torch
with open(f'./models/{test_case}/x1_traversal_poisson2d.pkl', 'rb') as f:
    x1_traversal = pickle.load(f)
with open(f'./models/{test_case}/x2_traversal_poisson2d.pkl', 'rb') as f:
    x2_traversal = pickle.load(f)
# replace the nxexpr with x2_traversal
new_traversal = []
for token in x1_traversal:
    if token.name == 'Nxexpr':
        new_traversal.extend(x2_traversal)
    else:
        new_traversal.append(token)
print(new_traversal)
ini_consts = []
for token in new_traversal:
    if token.name == 'const':
        ini_consts.append(token.value)
ini_consts = [torch.tensor(i, requires_grad=True) for i in ini_consts]

[add, sub, 1e-08, sin, mul, sub, sub, add, x2, mul, x2, div, sub, mul, x2, x2, x2, x2, x2, mul, x2, x2, x2, sin, mul, div, x1, x1, x1]


In [8]:
# samples for the domain and boundary for refine
n_samples, n_boundary = 100, 100
X = np.random.uniform(left_bc, right_bc, (n_samples, 2))
X_bc = rect(xmin=[left_bc, left_bc],
         xmax=[right_bc, right_bc],
         n=n_samples,
         method='uniform')
X_bc_temp = X_bc.transpose(1, 0)
X_combine = np.concatenate([X, X_bc], axis=0)
X_combine_torch = torch.tensor(X_combine, dtype=torch.float32, requires_grad=True)
X_com_temp = X_combine.transpose(1, 0)
y = source_func(*X_com_temp).reshape(-1, 1)
y_bc = solution_func(*X_bc_temp).reshape(-1, 1)
y_input = [y, y_bc]
y_input_torch = [torch.tensor(i, dtype=torch.float32, requires_grad=True) for i in y_input]

from ssde.const import make_const_optimizer
from ssde.program import Program
from ssde.execute import python_execute as pe
from ssde.pde import function_map
from ssde.task.recursion.recursion import make_regression_metric


consts_index = [i for i in range(len(new_traversal)) if new_traversal[i].name == 'const']
metric,_,_ = make_regression_metric("neg_smse_torch", y_input)
def pde_r(consts):
    for i in range(len(consts)):
        new_traversal[consts_index[i]].torch_value = consts[i]
    y = pe(new_traversal, X_combine_torch)
    f = function_map['poisson2d'](y, X_combine_torch)
    y_hat = [f, y[-n_boundary:,0:1]]
    r = metric(y_input_torch,y_hat)
    obj = -r
    return obj

optimized_consts, smse = make_const_optimizer('torch')(pde_r, ini_consts)
for i in new_traversal:
    if i.name == 'const' or i.name == 'Nxexpr':
        i.parse_value()
test_p = Program()
test_p.traversal = new_traversal
sym_expr = sp.simplify(sp.N(sp.expand(sp.sympify(test_p.sympy_expr)),4))
print(f'Identified solution: {sym_expr}')


Identified solution: sin(x1) + sin(x2**2)
